Image classification with CNNs
================

The goal of this exercise is to implement a specific CNN architecture with PyTorch and train it on the CIFAR-10 image classification dataset. We will start by introducing the dataset and then implement a `nn.Module` and a useful `Solver` class. Seperating the model from the actual training has proven itself as a sensible design decision. By the end of this exercise you should have succesfully trained your (possible) first CNN model and have a boilerplate `Solver` class which you can reuse for the next exercise and your future research projects.

For an inspiration on how to implement a model or the solver class you can have a look at [these](https://github.com/pytorch/examples) PyTorch examples.

In [14]:
import numpy as np
import os
from random import choice
from string import ascii_uppercase
import torch
from torch.autograd import Variable
from yz.data_utils_harddisk import get_Cancer_datasets
from yz.solver import Solver
from yz.data_utils_harddisk import get_balanced_weights
from torchvision import models
import torch.nn as nn
import pandas as pd
from bayes_opt import BayesianOptimization
from tqdm import tqdm

csv_full_name = '~/dl4cvproject/data/train.csv'
img_folder_full_name = '~/dl4cvproject/data/train256'
csv_full_name = os.path.expanduser(csv_full_name)
img_folder_full_name = os.path.expanduser(img_folder_full_name)

csv_full_name_test = '~/dl4cvproject/data/test.csv'
img_folder_full_name_test = '~/dl4cvproject/data/test256'
csv_full_name_test = os.path.expanduser(csv_full_name_test)
img_folder_full_name_test = os.path.expanduser(img_folder_full_name_test)

#%matplotlib inline
#plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
#plt.rcParams['image.interpolation'] = 'nearest'
#plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
train_data, val_data, test_data, train_label_list = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name)
test_X, csv_test = get_Cancer_datasets(csv_full_name=csv_full_name_test,img_folder_full_name=img_folder_full_name_test, mode='upload')
print("Train size: %i" % len(train_data))
print("Val size: %i" % len(val_data))
print("Test size: %i" % len(test_data))
print("upload size: {}".format(len(test_X)))


100%|██████████| 18577/18577 [00:00<00:00, 916925.58it/s]

num_training:13000
num_validation:1857
num_test:3720
OK...
Train size: 13000
Val size: 1857
Test size: 3719
upload size: 12386


In [16]:
if torch.cuda.is_available():
    print('Cuda available')
else:
    print('Cuda not available :(---(')

Cuda available


In [17]:
def target(factor, batch_size, lr_const, lr_exp, weight_decay_const, weight_decay_exp, num_epochs):

    batch_size = int(batch_size)
    num_epochs = int(num_epochs)
    lr_const = int(lr_const)
    weight_decay_const = int(weight_decay_const)
    
    #training
    weights = get_balanced_weights(label_list=train_label_list, num_classes=14, factor=factor)
    sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=sampler, num_workers=8)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=20, shuffle=False, num_workers=8)
    
    model = models.resnet34(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, 14)

    lr = lr_const * np.power(10, lr_exp)
    weigth_decay = weight_decay_const * np.power(10, weight_decay_exp)    
    solver = Solver(optim_args={"lr":lr, "weight_decay":weigth_decay})
    solver.train(model, train_loader, val_loader, log_nth=50, num_epochs=num_epochs)
    
    #compute local prediction acc
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=30, shuffle=False, num_workers=8)
    scores = []
    for inputs, target in tqdm(test_loader):
        inputs, targets = Variable(inputs), Variable(target)
        if torch.cuda.is_available:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        scores.extend((preds == targets).data.cpu().numpy())
        
    test_acc = np.mean(scores)
    print(test_acc)
    
    ## generate submission file: submissions/res18_acc_randomsuffix.csv
    if test_acc >= 0.32:
        
        file_name = 'submissions/res18_' + '{:.5f}'.format(test_acc) + '_' + ''.join(choice(ascii_uppercase) for i in range(7)) + '.csv'
        print(file_name)
        
        try:
            del csv_test['age']
        except KeyError as e:
            print(e)
        try:
            del csv_test['gender']
        except KeyError as e:
            print(e)
        try:
            del csv_test['view_position']
        except KeyError as e:
            print(e)
        try:
            del csv_test['image_name']
        except KeyError as e:
            print(e)
        try:
            del csv_test['detected']
        except KeyError as e:
            print(e)

        pred_set = set()
        detected = []
        for i in tqdm(range(len(test_X))):
            tmp_pred_list = [0] * 14
            inputs = test_X[i]
            inputs = Variable(inputs.unsqueeze(0))
            if torch.cuda.is_available:
                inputs = inputs.cuda()
            for trial in range(1):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                int_label = preds.data.cpu().numpy().tolist()[0]
                tmp_pred_list[int_label] += 1

            pred = tmp_pred_list.index(max(tmp_pred_list))
            str_pred = 'class_' + str(pred + 1)
            detected.append(str_pred)

        csv_test['detected'] = pd.Series(detected)
        csv_test.to_csv(file_name, index=False)
    
    return test_acc

    

## Bayesian Optimization

In [18]:
bo = BayesianOptimization(target, {'factor':(0.5, 1), 'batch_size':(48, 48),
                                   'lr_const':(1, 10), 'lr_exp':(-7, -3),
                                   'weight_decay_const':(1, 10), 'weight_decay_exp':(-3, 0),
                                   'num_epochs':(6, 12)})

In [ ]:
bo.maximize(init_points=2, n_iter=1000, acq='ucb', kappa=5)

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   batch_size |    factor |   lr_const |    lr_exp |   num_epochs |   weight_decay_const |   weight_decay_exp | 
weights: [0.035190068364805649, 0.026902012132507265, 0.0067017071831152551, 0.017723868297630941, 0.011870579412525737, 0.017140457632990248, 0.011427364695711222, 0.037809570653116131, 0.11360786557948216, 0.057561333003794839, 0.023814003249156845, 0.01465198545619238, 0.026140473790401037, 0.030627831927392719]
equivalent_num:
10.6625907145
12.374925581
26.7465133678
15.5970041019
19.479620816
15.8892042258
19.8950419352
10.246393647
5.56678541339
8.11614795354
13.2405858065
17.3332987947
12.5735678932
11.5160648047



  0%|          | 0/9 [00:00<?, ?it/s]

START TRAIN.











Exception in thread Thread-181:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



[Iteration 1 / 2439] TRAIN loss: 2.774244
[Iteration 51 / 2439] TRAIN loss: 2.832072
[Iteration 101 / 2439] TRAIN loss: 2.617461
[Iteration 151 / 2439] TRAIN loss: 2.869205
[Iteration 201 / 2439] TRAIN loss: 2.725836
[Iteration 251 / 2439] TRAIN loss: 2.670400
[Epoch 1 / 9] TRAIN acc: 3.243542


...

[Epoch 3 / 8] VAL acc: 6.053763
[Iteration 851 / 2168] TRAIN loss: 2.635105
[Iteration 901 / 2168] TRAIN loss: 2.635482
[Iteration 951 / 2168] TRAIN loss: 2.635577
[Iteration 1001 / 2168] TRAIN loss: 2.634423
[Iteration 1051 / 2168] TRAIN loss: 2.633999
[Epoch 4 / 8] TRAIN acc: 8.022140


 50%|█████     | 4/8 [13:30<13:30, 202.60s/it]

[Epoch 4 / 8] VAL acc: 6.053763
[Iteration 1101 / 2168] TRAIN loss: 2.635971
[Iteration 1151 / 2168] TRAIN loss: 2.633462
[Iteration 1201 / 2168] TRAIN loss: 2.636598
[Iteration 1251 / 2168] TRAIN loss: 2.633398
[Iteration 1301 / 2168] TRAIN loss: 2.635916
[Iteration 1351 / 2168] TRAIN loss: 2.635305
[Epoch 5 / 8] TRAIN acc: 7.848708


 62%|██████▎   | 5/8 [16:46<10:03, 201.24s/it]

[Epoch 5 / 8] VAL acc: 6.053763
[Iteration 1401 / 2168] TRAIN loss: 2.633165
[Iteration 1451 / 2168] TRAIN loss: 2.634129
